In [143]:
import numpy as np
import random as rd
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button, Slider
import matplotlib.cm as cm
import scipy as SP

import threading

In [144]:
class Environment():
    
    #fields:
    #a grid of size defined by parameter
    #b, a tuple repsenting the delivery location
    #a, a tuple representing the package location
    #size, an int representing the horizontal and vertical dimensions of the grid
    def __init__(self, size:int, a = None):
        self.size = size
        self.b = (size-1, size-1)
        self.grid, self.a = self.setup_grid(size, a)
        self.reward_pickup = 100
        self.reward_deliver = 100
        self.reward_move = -1
        ## IS THIS WHERE THE AGENT SHOULD BE INITIATED?

    #method to set up the original grid including a location
    def setup_grid(self, size, a):
        grid = np.zeros((size,size))
        if a is None:
            x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            while (x,y) == self.b:
                x, y = rd.randint(0, size-1), rd.randint(0, size-1)
            grid[x,y] = 1  #A represented by 1
        else:
            x,y=a
            grid[x,y] = 1
        grid[size-1,size-1] = 2  #B represented by 2
        return grid, (x,y)
    
    #method to add start location of agent to grid as well
 #TODO: consider whether to initialise agent as part of environment initialisation
 #pass agent coordinates into constructor instead lmao DENSE
    def set_agent_start(self, agent):
        self.agent_coords = (agent.x, agent.y)
    
    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, agent_move):
        x, y = self.agent_coords
        self.grid[x,y] = 0
        self.agent_coords = agent_move[0] , agent_move[1]
        self.grid[agent_move[0] , agent_move[1]] = -1
        
    def plot_grid(self):
    #Create a figure and axis with a white background
        fig, ax = plt.subplots()
        ax.set_facecolor('white')  # Set background color to white

        # Plot the grid
        ax.imshow(np.array([[0]]), cmap="bone", extent=[0, self.size, 0, self.size])
        for i in range(self.size):
            for j in range(self.size):
                ax.text(j + 0.5, self.size - i - 0.5,self.grid[i,j],  ha='center', va='center', fontsize=20, color='white')

        # Set axis properties
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_xticks(np.arange(self.size) + 1)
        ax.set_yticks(np.arange(self.size) + 1)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True, linewidth=2, color='white')

        # Set title
        ax.set_title("5x5 Grid Visualization")

        #Show the plot
        plt.show()

#An agent class. Currently barebones
class Agent():
    def __init__(self, environ):
        self.environ = environ
        self.reset_agent()
        self.opts = ["up", "down", "left", "right"]
        

    # choose the best action based on the greedy policy on Q-values
    # filed qmat = Q-matrix of size n*n*options where n=grid_size, options=actions
    def choose_best_option(self, qmat):
        if self.collected == 0:
            state_qs = qmat[self.x-self.environ.a[0] + self.environ.size-1,
                            self.y-self.environ.a[1] + self.environ.size-1,
                            self.collected] # 1*1*options
        else:
            state_qs = qmat[self.x-self.environ.b[0] + self.environ.size-1,
                            self.y-self.environ.b[1] + self.environ.size-1,
                            self.collected] # 1*1*options
        action = np.argmax(state_qs) # choose action based on greedy policy where 0=up, 1=down, 2=left, 3=right
        return action

    #Ensure that moves
    def move(self, direction):

        if direction=="up":
            if self.x-1 >= 0:
                self.x = self.x-1
        if direction=="down":
            if self.x+1 < self.environ.size:
                self.x = self.x+1
        if direction=="left":
            if self.y-1 >= 0:
                self.y = self.y-1
        if direction=="right":
            if self.y+1 < self.environ.size:
                self.y=self.y+1
        
        if self.collected == 0:
            if (self.x, self.y) == self.environ.a:
                self.collected = 1
                reward = self.environ.reward_pickup
            else:
                reward = self.environ.reward_move
        
        else:
            if (self.x, self.y) == self.environ.b:
                self.end_episode = 1
                reward = self.environ.reward_deliver
            else:
                reward = self.environ.reward_move

        return (self.x, self.y, self.collected), reward, self.end_episode
    
    def reset_agent(self):
        self.x, self.y = rd.randint(0, self.environ.size-1), rd.randint(0, self.environ.size-1)
        while ((self.x, self.y) == (self.environ.a or self.environ.b)):
            self.x, self.y = rd.randint(0, self.environ.size-1), rd.randint(0, self.environ.size-1)
        self.collected = 0
        self.end_episode = 0

    
#Class to store hyperparameters
class Hypers():
    def __init__(self, eps, gamma, alpha, steps_per_ep, total_iter):
        self.eps = eps
        self.gamma = gamma
        self.alpha = alpha
        self.steps_per_ep = steps_per_ep
        self.total_iter = total_iter

##Method to initiate agent coordinates should go in setup method since both the Environment and the Agent need to know
#Or we need a hierarchy - consult w/ group

## Q Learn Notes

Rewards come from the environment, agent has values on states that it figures out from the rewards that the environment offers it.

Game has multiple episodes (like, tens of thousands).

Episode has multiple (hundreds?) timesteps. At each episode, we'll have a different location of A from all 24 possible states.

Q table learned across all 24 A locations, 25 agent locations, 4 possible actions, and the pre/post plant situation.

So in total 24 * 25 * 2 states, and four possible actions in each state.

4800 state-action space in total for this toy example.

#### SPECULATIVELY;

If we're constructing this Q-table and using the Bellman equation to backpropagate the rewards from end-state into the Q table, we are effectively "learning" a table which tells us what future rewards are going to be in future states. This lets us, in our _current_ state, choose the action which will best be producing future rewards. So our first near empty Q table will have the agent wander around effectively randomly.


Start with a fixed A and fixed B, to see if you CAN actually learn the Q values. Only THEN do we change to a dynamic location of A.

In [145]:
# Demo

hypers = Hypers(
    eps = 0.4, 
    gamma = 0.8,
    alpha = 0.01,
    steps_per_ep = 100,
    total_iter = 50000
)

import random as rd

env = Environment(size=5)
bond = Agent(env)
env.set_agent_start(bond)


qmat = np.zeros([2*(env.size-1)+1 , 2*(env.size-1)+1 , 2 , len(bond.opts)])

for i in range(hypers.total_iter):
    # print("qmat = ", np.argmax(qmat,3))
    env = Environment(size=5)
    env.set_agent_start(bond)
    bond = Agent(env)
    bond.reset_agent()
    
    # print("pickup - a: ",env.a)
    # print("agent - pos: ",bond.x,bond.y)
    
    for s in range(hypers.steps_per_ep):

        # choose between explore and exploit (epsilon-greedy)
        if rd.random()<hypers.eps:
            action = rd.randint(0,len(bond.opts)-1)
        else:
            action = bond.choose_best_option(qmat)


        if bond.collected == 0:
            state = (bond.x-env.a[0] + env.size-1 , bond.y-env.a[1] + env.size-1 , bond.collected)
        else:
            state = (bond.x-env.b[0] + env.size-1 , bond.y-env.b[1] + env.size-1 , bond.collected)

        new_agent_pos, reward, end_episode = bond.move(bond.opts[action])
        if bond.collected == 0:
            next_state = (new_agent_pos[0]-env.a[0] + env.size-1 , new_agent_pos[1]-env.a[1] + env.size-1 , bond.collected)
        else:
            next_state = (new_agent_pos[0]-env.b[0] + env.size-1 , new_agent_pos[1]-env.b[1] + env.size-1 , bond.collected) 
        
        env.move_agent(new_agent_pos)
        
        next_action = np.argmax(qmat[next_state])
        qmat[state][action]+=hypers.alpha*(reward + (hypers.gamma*(qmat[next_state][next_action])) - qmat[state][action])
        

        # print("s, reward: ", s, reward)
        if end_episode == 1:
            # print("qmat = ", np.argmax(qmat,3))
            # print("end of episode")
            break


# for i in range(10):
#     env.move_agent(bond.move(rd.choice(bond.opts)))
#     print(env.grid)
#     env.plot_grid()
#     time.sleep(1)


# class QL():
#     def __init__(self, environ, agent):

In [166]:
env = Environment(size = 5) #, a = (0,4)
bond = Agent(env)
env.set_agent_start(bond)

# bond.x = 4
# bond.y = 0
env.move_agent((bond.x, bond.y, bond.collected))
# env.plot_grid()
for step in range(20):
    
    if bond.collected == 0:
        state = (bond.x-env.a[0] + env.size-1 , bond.y-env.a[1] + env.size-1 , bond.collected)
    else:
        state = (bond.x-env.b[0] + env.size-1 , bond.y-env.b[1] + env.size-1 , bond.collected)

    action = bond.choose_best_option(qmat)
    
    pos,_,end = bond.move(bond.opts[action])
    env.move_agent(pos) 
    # print(env.grid)
    # env.plot_grid()
    if end == 1:
        print("end")
        break

end
